In [1]:
import os
import random

import keras
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.datasets import fashion_mnist
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Reproducible results...

# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape)

Using TensorFlow backend.


(60000, 28, 28)


In [2]:
# Split data into train and validation sets (80% and 20% respectively)
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2,random_state=12345)

In [3]:
# Define number of classes, image shape and batch size
num_classes = 10
image_rows = 28
image_cols = 28
batch_size = 512
image_shape = (image_rows, image_cols, 1)

In [4]:
# Reshape dataset
x_train = x_train.reshape(x_train.shape[0], *image_shape)
x_test = x_test.reshape(x_test.shape[0], *image_shape)
x_validate = x_validate.reshape(x_validate.shape[0], *image_shape)

# TODO: try normalization

In [5]:
# Define some models

name = '1_Layer'
cnn_model_1 = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', input_shape=image_shape, name='Conv2D-1'),
    MaxPooling2D(pool_size=2, name='MaxPool'),
    Dropout(0.2, name='Dropout'),
    Flatten(name='flatten'),
    Dense(32, activation='relu', name='Dense'),
    Dense(10, activation='softmax', name='Output')
], name=name)

name = '2_Layer'
cnn_model_2 = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', input_shape=image_shape, name='Conv2D-1'),
    MaxPooling2D(pool_size=2, name='MaxPool'),
    Dropout(0.2, name='Dropout-1'),
    Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2'),
    Dropout(0.25, name='Dropout-2'),
    Flatten(name='flatten'),
    Dense(64, activation='relu', name='Dense'),
    Dense(10, activation='softmax', name='Output')
], name=name)

name='3_layer'
cnn_model_3 = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', input_shape=image_shape, kernel_initializer='he_normal', 
           name='Conv2D-1'),
    MaxPooling2D(pool_size=2, name='MaxPool'),
    Dropout(0.25, name='Dropout-1'),
    Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2'),
    Dropout(0.25, name='Dropout-2'),
    Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3'),
    Dropout(0.4, name='Dropout-3'),
    Flatten(name='flatten'),
    Dense(128, activation='relu', name='Dense'),
    Dropout(0.4, name='Dropout'),
    Dense(10, activation='softmax', name='Output')
], name=name)

cnn_model = Sequential([
    Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=image_shape),
    MaxPooling2D(pool_size=2),# down sampling the output instead of 28*28 it is 14*14
    Dropout(0.2),
    Flatten(), # flatten out the layers
    Dense(32,activation='relu'),
    Dense(10,activation = 'softmax')
    
])

cnn_models = [cnn_model_1, cnn_model_2, cnn_model_3]

W1012 14:01:04.029224 140478202677056 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1012 14:01:04.037150 140478202677056 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1012 14:01:04.076004 140478202677056 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4255: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1012 14:01:04.084139 140478202677056 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1012 14:01:04.085345 140478202677056 deprecation_wrap

In [ ]:
# Fit models
history_dict = {}
for model in cnn_models:
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(), # Adam(lr=0.001),
        metrics=['accuracy']
    )
    
    history = model.fit(
        x_train, y_train,
        batch_size=batch_size,
        epochs=50, verbose=1,
        validation_data=(x_validate, y_validate)
    )
    
    history_dict[model.name] = history

W1012 14:01:18.080354 140478202677056 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1012 14:01:18.197843 140478202677056 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 30s 632us/step - loss: 6.1264 - acc: 0.6019 - val_loss: 4.0672 - val_acc: 0.7211
Epoch 2/50
48000/48000 [==============================] - 30s 624us/step - loss: 1.4495 - acc: 0.7844 - val_loss: 0.4418 - val_acc: 0.8572
Epoch 3/50
48000/48000 [==============================] - 27s 572us/step - loss: 0.4256 - acc: 0.8564 - val_loss: 0.3722 - val_acc: 0.8768
Epoch 4/50
48000/48000 [==============================] - 30s 623us/step - loss: 0.3538 - acc: 0.8748 - val_loss: 0.3445 - val_acc: 0.8854
Epoch 5/50
48000/48000 [==============================] - 27s 566us/step - loss: 0.3142 - acc: 0.8861 - val_loss: 0.3252 - val_acc: 0.8908
Epoch 6/50
48000/48000 [==============================] - 27s 568us/step - loss: 0.2848 - acc: 0.8953 - val_loss: 0.3193 - val_acc: 0.8912
Epoch 7/50
48000/48000 [==============================] - 24s 502us/step - loss: 0.2657 - acc: 0.9015 - val_loss: 0.

In [ ]:
# Get test loss and accuracy
score = cnn_model.evaluate(x_test, y_test, verbose=0)
print('Test Loss : {:.4f}'.format(score[0]))
print('Test Accuracy : {:.4f}'.format(score[1]))

In [ ]:
# Plot losses and accuracies by epoch
fig,(ax1, ax2)=plt.subplots(2, figsize=(8, 6))
for history in history_dict:
    val_acc = history_dict[history].history['val_acc']
    val_loss = history_dict[history].history['val_loss']
    ax1.plot(val_acc, label=history)
    ax2.plot(val_loss, label=history)

ax1.set_ylabel('Validation Accuracy')
ax2.set_ylabel('Validation Loss')
ax1.set_xlabel('Epochs')
ax1.legend()
ax2.legend()
plt.show()  

In [ ]:
cnn_model.summary()